In [30]:
import os
# import pandas as pd
# import psycopg2
# import sqlalchemy as sqla
# from dotenv import load_dotenv, find_dotenv
# load_dotenv(find_dotenv())
import sql as gd

In [2]:
print(os.getenv('password'))

64CjIlzDWuV9BivO


In [34]:
# # join airports with flights_faso
# all = gd.get_data('SELECT * FROM flights_faso INNER JOIN airports on airports.faa=flights_faso.origin')
# all.head()
# fa_selected = all.query('faa == "DCA" or faa == "IAD" or faa == "LGA" or faa == "JFK"')
# # ['JFK', 'IAD', 'LGA', 'DCA']

points = gd.get_data("SELECT faa, lat,lon FROM airports WHERE faa in ('JFK', 'IAD', 'LGA', 'DCA')")
points


/opt/homebrew/Caskroom/miniconda/base/envs/sql-practice/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,faa,lat,lon
0,DCA,38.852100,-77.037697
1,LGA,40.777199,-73.872597
2,IAD,38.944500,-77.455803
3,JFK,40.639801,-73.778900


In [ ]:
api = '06cc1bc58bmsh6b0e697965add32p1e5320jsnf3103b779dfa'

curl --request GET \
	--url 'https://meteostat.p.rapidapi.com/stations/meta?id=10637' \
	--header 'x-rapidapi-host: meteostat.p.rapidapi.com' \
	--header 'x-rapidapi-key: {06cc1bc58bmsh6b0e697965add32p1e5320jsnf3103b779dfa}'

In [10]:
import requests

url = "https://meteostat.p.rapidapi.com/point/daily"

querystring = {"lat": "40.77719879","lon": "-73.87259674","start":"2012-10-29","end":"2012-10-29"}
# https://meteostat.p.rapidapi.com/point/daily?lat=43.6667&lon=-79.4&start=2020-01-01&end=2020-01-31&alt=184'
headers = {
	"X-RapidAPI-Host": "meteostat.p.rapidapi.com",
	"X-RapidAPI-Key": ""
}
response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)
type(response)

{"meta":{"generated": "2022-05-05 14:31:45", "stations": ["KNYC0", "72502", "KTEB0", "74486"]},"data":[{"date":"2012-10-29","tavg":14.5,"tmin":12.2,"tmax":17.8,"prcp":15.2,"snow":0.0,"wdir":35.0,"wspd":54.7,"wpgt":null,"pres":null,"tsun":null}]}


requests.models.Response

In [11]:
import json

print(json.dumps(response.json(), indent=2))

{
  "meta": {
    "generated": "2022-05-05 14:31:45",
    "stations": [
      "KNYC0",
      "72502",
      "KTEB0",
      "74486"
    ]
  },
  "data": [
    {
      "date": "2012-10-29",
      "tavg": 14.5,
      "tmin": 12.2,
      "tmax": 17.8,
      "prcp": 15.2,
      "snow": 0.0,
      "wdir": 35.0,
      "wspd": 54.7,
      "wpgt": null,
      "pres": null,
      "tsun": null
    }
  ]
}


In [ ]:
_df = pd.json_normalize(weather_hh, 
                                  sep="_", 
                                  record_path="weather", 
                                  meta=[["coord", "lon"], 
                                        ["coord", "lat"], 
                                        "base",
                                        ["main", "temp"],
                                        ["main", "feels_like"],
                                        ["main", "temp_min"],
                                        ["main", "temp_max"], 
                                        ["main", "pressure"], 
                                        ["main", "humidity"], 
                                        "visibility", 
                                        ["wind", "speed"], 
                                        ["wind", "deg"], 
                                        ["clouds", "all"], 
                                        "dt", 
                                        ["sys", "type"], 
                                        ["sys", "id"],
                                        ["sys", "country"],
                                        ["sys", "sunrise"],
                                        ["sys", "sunset"],
                                        "timezone",
                                        "id",
                                        "name",
                                        "cod"], 
                                  record_prefix="weather_")